In [132]:
import numpy as np
import glob
import time

#It's kk to import whatever you want from the local util module if you would like:
#from util.X import ... 
%pylab inline 
    # Load Image
data_dir = '../data/'

easy_images = glob.glob(data_dir + 'easy/*/*')
medium_images = glob.glob(data_dir + 'medium_1/*/*')
medium_images.extend(glob.glob(data_dir + 'medium_2/*/*'))
hard_images = glob.glob(data_dir + 'hard/ball/*')

tic = time.time()
im = imread('../data/easy/brick/brick_1.jpg')
# im = imread('../data/easy/ball/ball_2.jpg')
    
    
    
def convert_to_grayscale(im):
    '''
    Convert color image to grayscale.
    Args: im = (nxmx3) floating point color image scaled between 0 and 1
    Returns: (nxm) floating point grayscale image scaled between 0 and 1
    np.all(im == [255, 0, 0], axis = -1)
    '''
    return np.mean(im, axis = 2)

def roberts_cross(x):
    '''Compute Robert's Cross of input image x.
       Args: x (nxm) grayscale floating point image
       Returns: (n-1) x (m-1) edge image.'''
    
    #Our output image will be one pixel smaller than our image
    edges = np.zeros((x.shape[0]-1,x.shape[1]-1)) 

    for i in range(x.shape[0]-1):
        for j in range(x.shape[1]-1):
            #Grab Appropriate (2x2) image patch
            image_patch = x[i:i+2, j:j+2]
            # Compute Robert's Cross for image patch
            edges[i, j] = np.sqrt((image_patch[0,0] - image_patch[1, 1])**2 + 
                                   (image_patch[1, 0] - image_patch[0, 1])**2)
            
    return edges
def filter_2d(im, kernel):
    '''
    Filter an image by taking the dot product of each 
    image neighborhood with the kernel matrix.
    Args:
    im = (H x W) grayscale floating point image
    kernel = (M x N) matrix, smaller than im
    Returns: 
    (H-M+1 x W-N+1) filtered image.
    '''

    M, N = kernel.shape
    H, W = im.shape
    filtered_image = np.zeros((H-M+1, W-N+1), dtype = 'float64')
    
    for i in range(filtered_image.shape[0]):
        for j in range(filtered_image.shape[1]):
            image_patch = im[i:i+M, j:j+N]
            filtered_image[i, j] = np.sum(np.multiply(image_patch, kernel))
            
    return filtered_image
def make_gaussian_kernel(size, sigma):
    '''
    Create a gaussian kernel of size x size. 
    Args: 
    size = must be an odd positive number
    sigma = standard deviation of gaussian in pixels
    Returns: A floating point (size x size) guassian kernel 
    ''' 
    #Make kernel of zeros:
    kernel = np.zeros((size, size))
    
    #Handle sigma = 0 case (will result in dividing by zero below if unchecked)
    if sigma == 0:
        return kernel 
    
    #Helpful for indexing:
    k = int((size-1)/2)
    
    for i in range(size):
        for j in range(size):
            kernel[i, j] = (1/(2*np.pi*sigma**2))*exp(-((i-k)**2 + (j-k)**2)/(2*sigma**2))
            
    return kernel


# by Robert's cross

def classify_Roberts_cross(im):
    '''
    Example submission for coding challenge. 
    
    Args: im (nxmx3) unsigned 8-bit color image 
    Returns: One of three strings: 'brick', 'ball', or 'cylinder'
    
    '''
    #Let's guess randomly! Maybe we'll get lucky.
#     labels = ['brick', 'ball', 'cylinder']
#     random_integer = np.random.randint(low = 0, high = 3)
    
#     return labels[random_integer]

# -------------------------------------------------------
    im_scaled= im/255.
    gray=convert_to_grayscale(im_scaled)
    kernel = (1/9)*np.ones((3,3))
    gaussian_kernel = make_gaussian_kernel(size=11,sigma=2)
    im_filtered = filter_2d(gray,gaussian_kernel)
#     y=gray
    edges = roberts_cross(im_filtered)
    fig = figure(0, (24,12))
    fig.add_subplot(1,2,1)
    imshow(gray, cmap = 'gray')

    fig.add_subplot(1,2,2)
    imshow(edges)
#     colorbar()
    title("Edges computed with Robert's Cross")

def classify_Sobel_Feldman(im):
    Kx = np.array([[1, 0, -1],
               [2, 0, -2],
               [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
               [0, 0, 0],
               [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)

    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    thresh = .55
    edges_and_angles = np.zeros(G_magnitude.shape)*np.NaN #Create empty array of NaNs
    #Replace pixels with gradient estimates above thresh with the direction of the gradient estimate:
    edges_and_angles[G_magnitude>thresh] = G_direction[G_magnitude>thresh] 
    fig = figure(0, (16,12))
#     imshow(edges_and_angles)
#     colorbar();
    cmap = matplotlib.cm.get_cmap('viridis')

    r = (np.max(G_direction) - np.min(G_direction)) #range
    direction_scaled = (G_direction - np.min(G_direction))/r

    rgba = cmap(direction_scaled) 
    rgb = rgba[:,:,:-1]

    grad_rep = np.repeat(np.expand_dims(G_magnitude, 2), 3, axis = 2)
    scaled_im = np.multiply(rgb, grad_rep)

#     fig = figure(0, (16,12))
#     imshow(scaled_im)
    hist(edges_and_angles.ravel(), 100); grid(1)
    print(edges_and_angles.shape)
#     print("Peaks "+str(find_peaks(edges_and_angles)))
    toc = time.time()
#     print(toc-tic:0.4f)
#     print(str(toc-tic)+"seconds")
    print(edges_and_angles)

def peakDetection(array):
    import math;
    rows = array.shape[0]
    cols = array.shape[1]
#     temp = [cols]
    temp =np.zeros(cols)
    for x in range(0, cols - 1):
        for y in range(0, rows -1):
            temp[x]=max(temp[x],array[x][y])
#     print(temp)  
    temp2= temp.nonzero()
    
# def peakfinder(d):
#     dh = hist(d,plot=False)
#     ins = dh[["intensities"]]
#     nbins = length(ins)
#     ss = which(rank(ins)%in%seq(from=nbins-2,to=nbins)) ## pick the top 3 intensities
#     return dh[["mids"]][ss]
# def find_peaks(a):
#     x = np.array(a)
#     max = np.max(x)
#     lenght = len(a)
#     ret = []
#     for i in range(lenght):
#         ispeak = True
#         if i-1 > 0:
#             ispeak &= (x[i] > 1.8 * x[i-1])
#         if i+1 < lenght:
#             ispeak &= (x[i] > 1.8 * x[i+1])

#         ispeak &= (x[i] > 0.05 * max)
#         if ispeak:
#             ret.append(i)
#     return ret

def findMode(G):
    temp=G.argmax(0)
    m = mean(temp)
    item = temp>m
    temp1 = []
    from collections import Counter 
    for i in range(0,item.size,5):
        n_num = item[i:i+5] 
        n = len(n_num) 
        data = Counter(n_num) 
        get_mode = dict(data) 
        mode = [k for k, v in get_mode.items() if v == max(list(data.values()))] 
        temp1.append(mode[0])
    return temp1


def countEdges(arr):
    count=0
    for i in range(len(arr)):
        print(arr[i])
        if i==0:
            if arr[i]==True:
                count=count+1
                
        else:
            if arr[i-1]==False & arr[i]==True:
                count=count+1
            
    return count 
    
def classify_hough(im):
    gray = convert_to_grayscale(im/255.)
    Kx = np.array([[1, 0, -1],
               [2, 0, -2],
               [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
               [0, 0, 0],
               [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)
    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    edges = G_magnitude > 0.25
    y_coords, x_coords = np.where(edges)
    fig = figure(0, (12,9))
    scatter(x_coords, y_coords, s = 5)
    grid(1)
    toc = time.time()
    print(str(toc-tic)+"seconds")
    
def findMode1(G):
    m = mean(G)
    print("Mean "+str(m))
    item = G>m
    return item
    
def classify_Sobel_Feldman1(im):
    Kx = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])

    Ky = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]])
    gray = convert_to_grayscale(im/255.)
    Gx = filter_2d(gray, Kx)
    Gy = filter_2d(gray, Ky)
    G_magnitude = np.sqrt(Gx**2+Gy**2)
    G_direction = np.arctan2(Gy, Gx)
    thresh = 0.50
    direction_of_edges=G_direction[G_magnitude>thresh]
    counts, bin_edges = np.histogram(direction_of_edges, bins=50)
#     (n, bins, patches) = np.histogram(direction_of_edges,50)
    print("Hist "+str(counts))
    counts.astype(int)
    boolArray = findMode1(counts)
    np_array = np.array(boolArray)
    count =np.count_nonzero(np_array[:-1] < np_array[1:])
    print("Peaks "+str(count))
    return count

# classify_hough(im)
# classify_Sobel_Feldman(im)
# fig = figure(0, (14,7))
for i in range(10):
    im=imread(hard_images[i])
    if(classify_Sobel_Feldman1(im) ==5):
#         fig.add_subplot(4,2,i+1)
#         imshow(im)
        print(" It's a Ball "+str(i))
toc = time.time()
print(str(toc-tic) + " Seconds") 

Populating the interactive namespace from numpy and matplotlib
Hist [302 216 146 131 169 131 157 182 241 149 256 638 328 332 232 169 153 116
  96  96 143 134 186 219 141 120 164 156 114 121  92 119 138 160 142 227
 615 357 211 280 179 155 172 152 184 138 212 268 331 291]
Mean 203.22
Peaks 5
 It's a Ball 0
Hist [ 378  292  324  386  236  341  908  693 1283  685  678 1534  509  403
  360  268  349  447  426  348  301  241  324  876  782  402  284  307
  335  200  284  728  450  721  720  647  901  523  480  381  272  348
  618  505  342  328  317  377  834  491]
Mean 503.34
Peaks 6
Hist [ 461  525  669  520  432  643  394  625  553  517  540  511  542  896
 1141  953  888 1150  757  600  499  405  447  511  524  485  596  637
  618  485  628  459  664  682  635  562  486  546  645 1084  839  683
 1101  911  923  670  577  601  785  725]
Mean 654.6
Peaks 5
 It's a Ball 2
Hist [517 340 205 210 311 257 134 147 249 600 622 616 522 523 372 308 230 198
 250 215 215 182 212 312 509 464 376 316 